In [28]:
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
from keras import models, layers
import seaborn as sns

filename = 'shampoo-three-year.csv'

In [30]:
dataset = pd.read_csv(filename)

In [34]:
dataset = dataset.dropna()

In [35]:
dataset.shape

(36, 2)

In [36]:
dataset.head()

,Month,Sales of shampoo over a three year period
0,1-01,266.0
1,1-02,145.9
2,1-03,183.1
3,1-04,119.3
4,1-05,180.3
